In [1]:
"""
Baseline for CIL project on road segmentation.
This simple baseline consits of a CNN with two convolutional+pooling layers with a soft-max loss
"""

import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
from PIL import Image
from random import shuffle
import time

import code

import tensorflow.python.platform

import numpy as np
import tensorflow as tf

import cil_helper as cil

from skimage.feature import greycomatrix, greycoprops

In [2]:
try:
    tf.app.flags.DEFINE_string('log_dir', '/tmp/tensorflow_lukas_balz',
                           """Directory where to write event logs """
                           """and checkpoint.""")
except: 
    print(tf.app.flags.FLAGS.log_dir)
    
FLAGS = tf.app.flags.FLAGS

In [3]:
TRAINING_SIZE = 50
VALIDATION_SIZE = 10  # Size of the validation set.
TEST_SIZE = 50
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 64 # 64
NUM_EPOCHS = 50
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 200
ERROR_STEP = 500

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 16
NUM_CHANNELS = 6

# all prediction size
ALL_PER = 500

In [4]:
data_dir = 'training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 
test_dir = "test/"

In [5]:
# returns all train images in rgb
def get_train_imgs():
    imgs = []
    for i in range(TRAINING_SIZE):
        imageid = "satImage_%.3d" % (i+1)
        image_filename = train_data_filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            raise Exception('File ' + image_filename + ' does not exist')
    return np.asarray(imgs)

In [6]:
def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
            list_patches.append(np.rot90(im_patch))
            list_patches.append(np.fliplr(im_patch))
    return list_patches

In [7]:
# Extract label images
def extract_labels(filename, num_images):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    gt_patches = [img_crop(gt_imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    labels = np.asarray([value_to_class(np.mean(data[i])) for i in range(len(data))])

    # Convert to dense 1-hot representation.
    return labels.astype(np.float32)

In [8]:
def make_patches(imgs):
    num_images = len(imgs)
    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    DIMS = 4
    N_PATCHES_PER_IMAGE = (IMG_WIDTH/IMG_PATCH_SIZE)*(IMG_HEIGHT/IMG_PATCH_SIZE)

    img_patches = [img_crop(imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))]

    return np.asarray(data)

In [9]:
# Assign a label to a patch v
def value_to_class(v):
    foreground_threshold = 0.75 # percentage of pixels > 1 required to assign a foreground label to a patch
    #ret#urn [1-v , v]
    df = np.sum(v)
    if df < foreground_threshold:
       return [0, 1]
    else:
       return [1, 0]

In [10]:
def load_preproc(save_dir):
    assert save_dir.endswith('/')
    return np.load(save_dir + 'preprocessed2.npy')

In [11]:
train_preproc = load_preproc(train_data_filename)
train_data = make_patches(train_preproc)

train_labels = extract_labels(train_labels_filename, TRAINING_SIZE)

In [12]:
print(train_data.shape)
#NUM_CHANNELS = 3
#train_data = train_data[:,:,:,0:NUM_CHANNELS]
#print(train_data.shape)

(187500, 16, 16, 6)


In [13]:
c0 = 0
c1 = 0
for i in range(len(train_labels)):
    if train_labels[i][0] > 0.5:
        c0 = c0 + 1 # foreground -> road
    else:
        c1 = c1 + 1 # background
print ('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 9849 c1 = 83901


In [14]:
sampling = False

In [15]:
print ('Balancing training data...')
min_c = min(c0, c1)
max_c = max(c0, c1)
idx0 = [i for i, j in enumerate(train_labels) if j[0] == 1]
idx1 = [i for i, j in enumerate(train_labels) if j[1] == 1]

if sampling: #sample smaller class
    if c0 < c1:
        while len(idx0) < c1:
            print(len(idx0))
            idx0.extend(idx0)
            print(len(idx0))
    else:
        while len(idx1)<c0:
            idx1.extend(idx1)
    lim = max_c
else: 
    lim = min_c
    
new_indices = idx0[0:lim] + idx1[0:lim]
train_data = train_data[new_indices,:,:,:]
train_labels = train_labels[new_indices]
    
print(len(new_indices))
print(train_data.shape)

Balancing training data...
19698
(19698, 16, 16, 6)


In [16]:
train_size = train_labels.shape[0]

In [17]:
# This is where training samples and labels are fed to the graph.
# These placeholder nodes will be fed a batch of training data at each
# training step using the {feed_dict} argument to the Run() call below.
train_data_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS),
    name='train_data')
train_labels_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, cil.NUM_LABELS),
    name='train_labels')
train_all_data_node = tf.constant(train_data)

In [18]:
# The variables below hold all the trainable weights. They are passed an
# initial value which will be assigned when when we call:
# {tf.initialize_all_variables().run()}
conv1_weights = cil.weight_variable([3, 3, NUM_CHANNELS, 64])
conv1_biases = cil.bias_variable([64])

conv2_weights = cil.weight_variable([3, 3, 64, 128])
conv2_biases = cil.bias_variable([128])

conv3_weights = cil.weight_variable([3, 3, 128, 256])
conv3_biases = cil.bias_variable([256])

conv4_weights = cil.weight_variable([3, 3, 256, 512])
conv4_biases = cil.bias_variable([512])

fc1_weights = tf.Variable(  # fully connected, depth 512.
    tf.truncated_normal([2048, 4096],
                        stddev=0.05,
                        seed=SEED))
fc1_biases = tf.Variable(tf.constant(0.1, shape=[4096]))
fc2_weights = tf.Variable(
    tf.truncated_normal([4096, cil.NUM_LABELS],
                        stddev=0.1,
                        seed=SEED))
fc2_biases = tf.Variable(tf.constant(0.1, shape=[cil.NUM_LABELS]))
conv1_weights_hist = tf.histogram_summary("conv1_weights", conv1_weights)
conv2_weights_hist = tf.histogram_summary("conv2_weights", conv2_weights)
conv1_biases_hist = tf.histogram_summary("conv1_biases", conv1_biases)
conv2_biases_hist = tf.histogram_summary("conv2_biases", conv2_biases)
fc1_weights_hist = tf.histogram_summary("fc1_weights", fc1_weights)
fc1_biases_hist = tf.histogram_summary("fc1_biases", fc1_biases)
fc2_weights_hist = tf.histogram_summary("fc2_weights", fc2_weights)
fc2_biases_hist = tf.histogram_summary("fc2_biases", fc2_biases)

In [19]:
# We will replicate the model structure for the training subgraph, as well
# as the evaluation subgraphs, while sharing the trainable parameters.
def model(data, train=False):
    """The Model definition."""
    conv = cil.conv2d(data, conv1_weights)
    relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
    pool = cil.max_pool_2x2(relu)

    conv2 = cil.conv2d(pool, conv2_weights)
    relu2 = tf.nn.relu(tf.nn.bias_add(conv2, conv2_biases))
    pool2 = cil.avg_pool_2x2(relu2)
    
    conv3 = cil.conv2d(pool2, conv3_weights)
    relu3 = tf.nn.relu(tf.nn.bias_add(conv3, conv3_biases))
    #pool3 = cil.max_pool_2x2(relu3)
    
    conv4 = cil.conv2d(relu3, conv4_weights)
    relu4 = tf.nn.relu(tf.nn.bias_add(conv4, conv4_biases))
    pool4 = cil.max_pool_2x2(relu4)

    # Reshape the feature map cuboid into a 2D matrix to feed it to the
    # fully connected layers.
    pool_shape = pool4.get_shape().as_list()
    reshape = tf.reshape(
        pool4,
        [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])
    # Fully connected layer. Note that the '+' operation automatically
    # broadcasts the biases.
    
    #print(pool4.get_shape())
    #print(reshape.get_shape())
    #print(fc1_weights.get_shape())
    
    fc1 = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases)
    if train:
        fc1 = tf.nn.dropout(fc1, 0.7, seed=SEED)
    fc2 = tf.matmul(fc1, fc2_weights) + fc2_biases

    return fc2

In [20]:
# Training computation: logits + cross-entropy loss.
logits = model(train_data_node, True) # BATCH_SIZE*NUM_LABELS
# print 'logits = ' + str(logits.get_shape()) + ' train_labels_node = ' + str(train_labels_node.get_shape())
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits, train_labels_node))
#loss = tf.reduce_mean(tf.square(logits - train_labels_node))
loss_summary = tf.scalar_summary('loss', loss)

In [21]:
all_params_node = [conv1_weights, conv1_biases, conv2_weights, conv2_biases, fc1_weights, fc1_biases, fc2_weights, fc2_biases]
all_params_names = ['conv1_weights', 'conv1_biases', 'conv2_weights', 'conv2_biases', 'fc1_weights', 'fc1_biases', 'fc2_weights', 'fc2_biases']
all_grads_node = tf.gradients(loss, all_params_node)
all_grad_norms_node = []
for i in range(0, len(all_grads_node)):
    norm_grad_i = tf.global_norm([all_grads_node[i]])
    all_grad_norms_node.append(norm_grad_i)
    all_params_summary = tf.scalar_summary(all_params_names[i], norm_grad_i)

In [22]:
# L2 regularization for the fully connected parameters.
regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))
# Add the regularization term to the loss.
loss += 5e-4 * regularizers

In [23]:
# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
batch = tf.Variable(0)
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.exponential_decay(
    0.01,                # Base learning rate.
    batch * BATCH_SIZE,  # Current index into the dataset.
    train_size,          # Decay step.
    0.95,                # Decay rate.
    staircase=True)
learning_rate_summary = tf.scalar_summary('learning_rate', learning_rate)

In [24]:
# Use simple momentum for the optimization.
#optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(loss, global_step=batch)

# try adam
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=batch)

#optimizer = tf.train.AdadeltaOptimizer(learning_rate).minimize(loss)

In [25]:
# Predictions for the minibatch, validation set and test set.
train_prediction = tf.nn.softmax(logits)
# We'll compute them only once in a while by calling their {eval()} method.
train_all_prediction = tf.nn.softmax(model(train_all_data_node[0:ALL_PER,:,:,:]))

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

In [26]:
s = tf.InteractiveSession()

In [27]:
# Get prediction overlaid on the original image for given input image
def get_prediction_with_overlay(img):
    
    img_prediction = get_prediction(img)
    oimg = cil.make_img_overlay(img[:,:,:3], img_prediction)

    return oimg

# Get a concatenation of the prediction and groundtruth for given input file
def get_prediction_with_groundtruth(img):
    
    img_prediction = get_prediction(img)
    cimg = cil.concatenate_images(img[:,:,:3], img_prediction)

    return cimg

# Get prediction for given input image 
def get_prediction(img):
    data = np.asarray(cil.img_crop(img, IMG_PATCH_SIZE, IMG_PATCH_SIZE))
    data = data[:,:,:,0:NUM_CHANNELS]
    data_node = tf.constant(data)
    output = tf.nn.softmax(model(data_node))
    output_prediction = s.run(output)
    img_prediction = cil.label_to_img(img.shape[0], img.shape[1], IMG_PATCH_SIZE, IMG_PATCH_SIZE, output_prediction)

    return img_prediction

In [28]:
if RESTORE_MODEL:
    # Restore variables from disk.
    saver.restore(FLAGS.train_dir + "/model.ckpt")
    print("Model restored.")

In [ ]:
# Run all the initializers to prepare the trainable parameters.
tf.initialize_all_variables().run()

# Build the summary operation based on the TF collection of Summaries.
summary_op = tf.merge_summary([loss_summary, learning_rate_summary, all_params_summary, conv1_weights_hist, conv2_weights_hist])
summary_writer = tf.train.SummaryWriter(FLAGS.log_dir,
                                                graph=s.graph)
print ('Initialized!')

Initialized!


In [ ]:
# Loop through training steps.
print ('Total number of iterations = ' + str(int(NUM_EPOCHS * train_size / BATCH_SIZE)))

training_indices = range(train_size)

for iepoch in range(NUM_EPOCHS):

    # Permute training indices
    perm_indices = np.random.permutation(training_indices)

    for step in range (int(train_size / BATCH_SIZE)):

        offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
        batch_indices = perm_indices[offset:(offset + BATCH_SIZE)]

        # Compute the offset of the current minibatch in the data.
        # Note that we could use better randomization across epochs.
        batch_data = train_data[batch_indices, :, :, :]
        batch_labels = train_labels[batch_indices]
        
        #print('batch_data.shape = ' + str(batch_data.shape))
        # This dictionary maps the batch data (as a numpy array) to the
        # node in the graph is should be fed to.
        feed_dict = {train_data_node: batch_data, train_labels_node: batch_labels}

        stepNr = step + iepoch*int(train_size / BATCH_SIZE)

        if stepNr % ERROR_STEP == 0:
            start_time = time.time()
            l, lr, all_predictions = s.run(
                        [loss, learning_rate, train_all_prediction],
                        feed_dict=feed_dict)
            
            duration = time.time() - start_time
            #print ('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
            #print ('Minibatch error: %.1f%%' % cil.error_rate(predictions,
                                                              #   batch_labels))
            loss_value = cil.error_rate(all_predictions, train_labels[:ALL_PER,:])
            
            format_str = ('step %d, error: %.1f%% (%.3f sec/batch)')
            print (format_str % (stepNr, loss_value, duration)) 
        elif stepNr % RECORDING_STEP == 0:
            
            start_time = time.time()
            l, lr, all_predictions = s.run(
                        [loss, learning_rate, train_prediction],
                        feed_dict=feed_dict)
            
            duration = time.time() - start_time
            #print ('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
            #print ('Minibatch error: %.1f%%' % cil.error_rate(predictions,
                                                              #   batch_labels))
            loss_value = cil.error_rate(predictions, batch_labels)
            
            format_str = ('step %d, batch error: %.1f%% (%.3f sec/batch)')
            print (format_str % (stepNr, loss_value, duration)) 
        else:
            # Run the graph and fetch some of the nodes.
            _, l, lr, predictions = s.run(
                        [optimizer, loss, learning_rate, train_prediction],
                        feed_dict=feed_dict)

Total number of iterations = 15389
step 0, error: 88.2% (0.896 sec/batch)


In [ ]:
save_path = saver.save(s, FLAGS.train_dir + "/model8.ckpt")

In [ ]:
print ("Running prediction on training set")
prediction_training_dir = "predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
    
for i in range(TRAINING_SIZE):
    pimg = get_prediction_with_groundtruth(train_preproc[i])
    Image.fromarray(pimg).save(prediction_training_dir + "prediction8_" + str(i+1) + ".png")
    oimg = get_prediction_with_overlay(train_preproc[i])
    oimg.save(prediction_training_dir + "overlay8_" + str(i+1) + ".png")  
print("Done predicting training data")

In [ ]:
print ("Running prediction on test set")
prediction_test_dir = "predictions_test/"
if not os.path.isdir(prediction_test_dir):
    os.mkdir(prediction_test_dir)

test_preproc = load_preproc(test_dir)
for i in range(TEST_SIZE):
    pred = get_prediction(test_preproc[i])
    pimg = cil.img_float_to_uint8(pred)
    Image.fromarray(pimg).save(prediction_test_dir + "prediction8_" + str(i+1) + ".png")
    oimg = cil.make_img_overlay(test_preproc[i], pred)
    oimg.save(prediction_test_dir + "overlay8_" + str(i+1) + ".png")
print("Done predicting test data")

In [ ]:
import re
th = 0.25

In [ ]:
# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > th:
        return 1
    else:
        return 0


def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    reg = re.search(r"\_\d+", image_filename)
    group = reg.group(0)[1:]
    img_number = int(group)
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))


def save(submission_filename):
    #subm#ission_filename = 'submission_07.csv'
    image_filenames = []
    for i in range(1, 51):
        imagename = 'prediction8_' + str(i)
        image_filename = 'predictions_test/' + imagename + '.png'
        print(image_filename)
        image_filenames.append(image_filename)
    masks_to_submission(submission_filename, *image_filenames)
    
save('submission_lukas_8x8.csv')